# Transfer Learning

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# 讀取MINST數據集

In [2]:
from tensorflow.keras.datasets import mnist

# 處理數據

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [5]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [6]:
#要把y改成one-hot encoding
from tensorflow.keras.utils import to_categorical

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 匯入需要套件

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D     #因為flitter是3*3矩陣 所以是2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam

# 建立模型

In [9]:
CNN_layers = [Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu', name='Conv_1'),
              MaxPooling2D(),
              Conv2D(128, (3, 3), padding='same', activation='relu', name='Conv_2'),
              MaxPooling2D(),
              Conv2D(512, (3, 3), padding='same', activation='relu', name='Conv_3'),
              GlobalAveragePooling2D()]

FC_layers = [Dense(units=256, activation='relu'),
             Dense(units=10, activation='softmax')]

In [10]:
model_1 = Sequential(CNN_layers+FC_layers)

In [11]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 128)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 512)         590336    
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [12]:
model_1.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [13]:
model_1.fit(x_train, y_train, batch_size= 128, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 297s 5ms/sample - loss: 0.5649 - categorical_accuracy: 0.8150
Epoch 2/2
60000/60000 [==============================] - 328s 5ms/sample - loss: 0.1593 - categorical_accuracy: 0.9521


# 讀取Fashion_MINST數據集

In [14]:
from tensorflow.keras.datasets import fashion_mnist

In [15]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [16]:
x_train.shape

(60000, 28, 28)

# 處理數據

## Channel
CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

(28,28) --> (28, 28, 1)

In [17]:
x_train = x_train.reshape(60000,28,28,1) /255

In [18]:
x_test = x_test.reshape(10000,28,28,1) /255

In [19]:
x_train[87].shape

(28, 28, 1)

In [20]:
y_train[87]

7

In [21]:
y_train = to_categorical(y_train, 10)

In [22]:
y_test = to_categorical(y_test, 10)

In [24]:
FC_layers_fashion = [Dense(64, activation='relu'),
                     Dense(32, activation='relu'), 
                     Dense(10, activation='softmax')]

In [25]:
model_2 = Sequential(CNN_layers+FC_layers_fashion)

# Frozen

In [26]:
for layer in CNN_layers:
    layer.trainable = False

In [27]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 128)       36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 512)         590336    
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [28]:
model_2.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [30]:
model_2.fit(x_train, y_train, batch_size= 128, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 51s 853us/sample - loss: 0.9039 - categorical_accuracy: 0.6763
Epoch 2/2
60000/60000 [==============================] - 59s 982us/sample - loss: 0.6611 - categorical_accuracy: 0.7538


In [31]:
score_train = model_2.evaluate(x_train, y_train)
score_test = model_2.evaluate(x_test, y_test)

print(f'Train Accuracy: {score_train[1]*100}')
print(f'Test Accuracy: {score_test[1]*100}')

10000/10000 [==============================] - 12s 1ms/sample - loss: 0.6345 - categorical_accuracy: 0.7739
Train Accuracy: 77.43666768074036
Test Accuracy: 77.38999724388123


# Fine-tune

In [32]:
for layer in CNN_layers:
    layer.trainable = True

In [ ]:
model_2.fit(x_train, y_train, batch_size= 128, epochs=2)

Train on 60000 samples
Epoch 1/2
11776/60000 [====>.........................] - ETA: 44s - loss: 0.6271 - categorical_accuracy: 0.7693

In [ ]:
score_train = model_2.evaluate(x_train, y_train)
score_test = model_2.evaluate(x_test, y_test)

print(f'Train Accuracy: {score_train[1]*100}')
print(f'Test Accuracy: {score_test[1]*100}')